In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

import cdsapi

In [ ]:
def snwe2str(snwe):
    """
    Get area extent in string
    https://github.com/insarlab/PyAPS/blob/244552cdfcf4e1a55de5f1439be4f08eb45872ec/src/pyaps3/utils.py#L23
    """
    if not snwe:
        return None
    s, n, w, e = snwe

    area = ''
    area += '_S{}'.format(abs(s)) if s < 0 else '_N{}'.format(abs(s))
    area += '_S{}'.format(abs(n)) if n < 0 else '_N{}'.format(abs(n))
    area += '_W{}'.format(abs(w)) if w < 0 else '_E{}'.format(abs(w))
    area += '_W{}'.format(abs(e)) if e < 0 else '_E{}'.format(abs(e))

    return area

In [ ]:
assert Path('~/.cdsapirc').expanduser().exists()

c = cdsapi.Client()

In [ ]:
humidparam = 'specific_humidity' #relative_humidity
date = '2020-01-01T00:00'
# Pressure levels
pressure_lvls = ['1','2','3','5','7','10','20','30','50',
                '70','100','125','150','175','200','225',
                '250','300','350','400','450','500','550',
                '600','650','700','750','775','800','825',
                '850','875','900','925','950','975','1000']

# Dictionary
year, month, day = date.split('-')[:3]
time = date.split('T')[-1]

model = 'ERA5'
snwe = [45, 46, -116, -115]
prefix_dict = {
            'ERA5'   : 'ERA5',
            'ERAINT' : 'ERAI',
            'HRES'   : 'HRES',
        }

filedir = Path('~/scratch/era5').expanduser()
fbase = Path(filedir).joinpath(prefix_dict[model])

flist = []
for day in [date]:
    fname = str(fbase)
    fname += snwe2str(snwe) if snwe is not None else ''
    fname += '_{}_{}.nc'.format(day, time)
    flist.append(fname)

humidparam = 'specific_humidity' #relative_humidity
date = pd.to_datetime('2020-01-01T00:00')
# Pressure levels
pressure_lvls = ['1','2','3','5','7','10','20','30','50',
                '70','100','125','150','175','200','225',
                '250','300','350','400','450','500','550',
                '600','650','700','750','775','800','825',
                '850','875','900','925','950','975','1000']

# Dictionary
year, month, day = date.year, date.month, date.day
time = date.strftime('%H%M')
indict = {'product_type'   :'reanalysis',
            'format'         :'netcdf',
            'variable'       :['geopotential','temperature', humidparam],
            'pressure_level' : pressure_lvls,
            'year'           : year,
            'month'          :month,
            'day'            :day,
            'time'           :time}

# download a geographical area subset
snwe = [45, 46, -116, -115]
if snwe is not None:
    s, n, w, e = snwe
    indict['area'] = '/'.join(['{:.2f}'.format(x) for x in [n, w, s, e]])

# Assert grib file not yet downloaded
if not Path(fname).exists():

    # Make the request
    c.retrieve('reanalysis-{}-pressure-levels'.format(model.lower()),indict,target=fname)
